In [ ]:
#GOOD LUCK
import re
import numpy as np
import pandas as pd
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
'''def pad_sequences(sequences, maxlen=2000):
    for s in sequences:
        if len(s) == 0: yield np.zeros((1, maxlen), dtype='int32')
        else:
          s = np.array(s, dtype='int32')
          yield np.expand_dims(np.pad(s[:maxlen], (0, max(0, maxlen - len(s))), 'constant'), axis=0)'''

In [ ]:
seed = 7
np.random.seed(seed)

In [ ]:
path_slang = "datafile/abusive_slang_stemmed.txt"
path_religious = "datafile/abusive_religious_stemmed.txt"
path_personal_attack = "datafile/abusive_personal_attack_stemmed.txt"
path_antifaminism = "datafile/abusive_antifaminism_stemmed.txt"
path_political =  "datafile/abusive_political_stemmed.txt"


def read_file(path):
    sent =[]
    sent.clear()
    with open(path, "r",encoding="utf-8") as f:
        for line in f:
            sent.append(line)
    return sent
slang_sent = read_file(path_slang)
religious_sent = read_file(path_religious)
personal_attack_sent = read_file(path_personal_attack)
political_sent = read_file(path_political)
antifaminism_sent = read_file(path_antifaminism)


In [ ]:
import re
def digit_remover(text):
    text = re.sub(r'[\d]','',text)
    text = re.sub(r'[?\!\t\‛\’\(\)\.\।]+','',text)
    text = re.sub(r'[/\-\,]',' ',text)
    text = text.rstrip("\n\r")
    return text


In [ ]:
def remover(sent):
    for i in range(len(sent)):
        sent[i] = digit_remover(sent[i])
    return sent
slang_sent = remover(slang_sent)
religious_sent = remover(religious_sent)
personal_attack_sent = remover(personal_attack_sent)
political_sent = remover(political_sent)
antifaminism_sent = remover(antifaminism_sent)


In [ ]:
slang_df = pd.DataFrame({'col':slang_sent})
slang_df['label'] = 0
religious_df = pd.DataFrame({'col':religious_sent})
religious_df['label'] = 1
personal_attack_df = pd.DataFrame({'col':personal_attack_sent})
personal_attack_df['label'] = 2
political_df = pd.DataFrame({'col':political_sent})
political_df['label'] = 3
antifaminism_df = pd.DataFrame({'col':antifaminism_sent})
antifaminism_df['label'] = 4

In [ ]:
frames = [slang_df, religious_df,personal_attack_df,political_df,antifaminism_df]
result = pd.concat(frames,ignore_index=True)
#suffle the data frame
result = result.sample(frac=1)

In [ ]:
from sklearn.model_selection import train_test_split
X = result['col']
y = result['label']
y = to_categorical(result['label'], num_classes=5)
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state = 42,test_size=.2)

In [ ]:
tokenize = Tokenizer()
tokenize.fit_on_texts(X)

In [ ]:
X = tokenize.texts_to_sequences(X)
#X_test = tokenize.texts_to_sequences(X_test)

In [ ]:
max_lenght = max([len(s.split()) for s in result['col']])
X = pad_sequences(X, max_lenght)
#X_test = pad_sequences(X_test, max_lenght)

# to cope up with variation of classes and small dataset

In [14]:
from imblearn.over_sampling import SMOTE
smote = SMOTE('minority')
X_sm,y_sm = smote.fit_sample(X_train,y_train)
print(X_sm.shape,y_sm.shape)

(2642, 124) (2642, 5)


# tune batch size and number of epoch

In [ ]:
def create_model():
    EMBEDDING_DIM = 100
    unknown = len(tokenize.word_index)+1
    model = Sequential()
    model.add(Embedding(unknown, EMBEDDING_DIM, input_length=max_lenght))
    model.add(LSTM(units=128, dropout=0.1, recurrent_dropout=0.2 ))
    model.add(Dense(5, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
from sklearn.model_selection import GridSearchCV


In [ ]:
multi_classes = ['ovr','crammer_singer']
batch_size = [64,100,128]
epochs = [7]
param_grid = dict(batch_size=batch_size, epochs=epochs)
model = KerasClassifier(build_fn=create_model, verbose=0)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid.fit(X, y)

In [ ]:
pd.DataFrame(grid.cv_results_)[['mean_test_score', 'std_test_score', 'params']]

In [ ]:
# examine the best model
print(grid.best_score_)
print(grid.best_params_)

In [30]:
#I choose to build 3 hidden layers
EMBEDDING_DIM = 100
unknown = len(tokenize.word_index)+1
model = Sequential()
model.add(Embedding(unknown, EMBEDDING_DIM, input_length=max_lenght))
model.add(LSTM(units=128, dropout=0.1, recurrent_dropout=0.2 ))
model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [31]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 124, 100)          532300    
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               117248    
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 645       
Total params: 650,193
Trainable params: 650,193
Non-trainable params: 0
_________________________________________________________________
None


In [32]:
print(y_train.shape)
print(X_train.shape)

(1998, 5)
(1998, 124)


In [47]:
#model.fit(X_sm, y_sm, batch_size=128, epochs=7, verbose=1)
model.fit(X_train, y_train, batch_size=128, epochs=7, verbose=1)

Epoch 1/7
1998/1998 [==============================] - 18s 9ms/step - loss: 0.0189 - acc: 0.9935
Epoch 2/7
1998/1998 [==============================] - 20s 10ms/step - loss: 0.0149 - acc: 0.9960
Epoch 3/7
1998/1998 [==============================] - 17s 9ms/step - loss: 0.0122 - acc: 0.9955
Epoch 4/7
1998/1998 [==============================] - 18s 9ms/step - loss: 0.0116 - acc: 0.9970
Epoch 5/7
1998/1998 [==============================] - 21s 10ms/step - loss: 0.0088 - acc: 0.9970
Epoch 6/7
1998/1998 [==============================] - 18s 9ms/step - loss: 0.0108 - acc: 0.9965
Epoch 7/7
1998/1998 [==============================] - 18s 9ms/step - loss: 0.0074 - acc: 0.9980


In [48]:
final_pred = model.predict_classes(X_test)
final_pred= to_categorical(final_pred)

In [49]:
# calculate accuracy of class predictions
from sklearn import metrics
metrics.accuracy_score(y_test, final_pred)

0.604

In [50]:
score,acc = model.evaluate(X_test,y_test,batch_size=128,verbose=1)
print(acc)

500/500 [==============================] - 1s 3ms/step
0.6040000033378601
